In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sb
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
plt.rcParams['figure.figsize'] = (20.0, 10.0)

In [ ]:
data_new = pd.read_csv('/kaggle/input/santander-pr/train.csv', dtype={"ind_ahor_fin_ult1":"int8",'ind_aval_fin_ult1':"int8", 'ind_cco_fin_ult1':"int8", 
                                                                  'ind_cder_fin_ult1':"int8", 'ind_cno_fin_ult1':"int8", 'ind_ctju_fin_ult1':"int8", 'ind_ctma_fin_ult1':"int8", 
                                                                  'ind_ctop_fin_ult1':"int8", 'ind_ctpp_fin_ult1':"int8",
                                                                  'ind_nomina_ult1':"float16", 'ind_nom_pens_ult1':"float16",
                                                                  'ind_actividad_cliente':"float16", 'cod_prov':"float16",
                                                                  'tipodom':"float16", "indrel":"float16", "ind_nuevo":"float16",
                                                                  'ind_deco_fin_ult1':"int8", 'ind_deme_fin_ult1':"int8", 'ind_dela_fin_ult1':"int8",
                                                                  'ind_ecue_fin_ult1':"int8", 'ind_fond_fin_ult1':"int8", 'ind_hip_fin_ult1':"int8",
                                                                  'ind_plan_fin_ult1':"int8", 'ind_pres_fin_ult1':"int8", 'ind_reca_fin_ult1':"int8", 'ind_tjcr_fin_ult1':"int8", 
                                                                  'ind_valo_fin_ult1':"int8", 'ind_viv_fin_ult1':"int8", 'ind_recibo_ult1':"int8"},
                      parse_dates=['fecha_dato', 'fecha_alta', 'ult_fec_cli_1t'],)

In [ ]:
pd.set_option('display.max_columns', 100)

For training purpose, we have 16 months of data starting from Jan 2015 upto April 2016



In [ ]:
data_new.isnull().sum()

27734, 16063 are very less in number. We could always delete them as they are meagre as compared to the number of records we have. Will try checking the accuracies in both the cases later on. For now, will replace the missing values.

The most people that are customers are youths.The age distribution is bimodal in nature i.e. there are 2 peaks one at the age interval 20-25 and other in the interval 40-50

In [ ]:
data_new['age'] = pd.to_numeric(data_new['age'], errors='coerce') # errors = 'coerce' will convert the rows having 'NA' to NaN

In [ ]:
mean_age = float(data_new['age'].mean())
mean_age = round(mean_age)
mean_age

In [ ]:
data_new['age'].fillna(value = mean_age, inplace = True)


In [ ]:
sb.distplot(data_new['age'], kde = True, bins = 100)
plt.title("Age Distribution")

In [ ]:
data_new['ind_nuevo'].value_counts()


The 'ind_nuevo' column indicates whether a customer is new or not.

0 indicates not a new customer

1 indicates a new customer.
Most of the customers aren't new
There are also NaN values. We need to replace the values with 1 or 0 depending on their relation with the bank.

Now for deciding what to fill the null values of ind_nuevo, let's check a few more columns like fecha_dato which gives when has the customer taken the contract, ind_actividad_cliente to check whether the customer is active or not

In [ ]:
x = data_new[['fecha_dato', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'ind_actividad_cliente', 'indfall', 'ind_empleado']].loc[data_new['ind_nuevo'].isna()]
x

There are 27734 customers about whom we don't know yet whether they are new or old customers. We need to know for how many months they have been associated with the Bank.

Moreover, all of them have been inactive in the Bank. We also don't know whether they are deceased or not as the *indfall column is also containing null for these rows.

These customers only have ind_cco_fin_ult1 ie Current Accounts(mostly used for business) and ind_reca_fin_ult1 ie Taxes Account. But the ind_empleado column which indicates they are still employed or not is also null. So we can proceed to drop these rows.



In [ ]:
#data_new.drop(data_new[data_new['ind_nuevo'].isna()].index, inplace = True)


In [ ]:
data_new['ind_empleado'].value_counts()

In [ ]:
data_new['pais_residencia'].value_counts().plot(x=None, y=None, kind='pie')

'ES' is the most frequent residential area. Replacing all he missing values by 'ES'.

In [ ]:
city_count  = data_new['ind_empleado'].value_counts()
city_count = city_count[:,]
sb.barplot(city_count.index, city_count.values, alpha=0.8)

#Employee index: A active, B ex employed, F filial, N not employee, P pasive
#most of the customers are not an employee of the bank

In [ ]:
city_count = data_new['sexo'].value_counts()
city_count = city_count[:,]
sb.barplot(city_count.index, city_count.values, alpha=0.8)

In [ ]:
data_new['fecha_dato_yearmonth'] = data_new['fecha_dato'].apply(lambda x: (100 * x.year) + x.month)

In [ ]:
month_year_value_counts = data_new['fecha_dato_yearmonth'].value_counts()


From the barplot, we can observe that till June 2015, there were almost constant number of customers ranging upto 6lac, but from July 2017, there is a sudden surge in the number of customers

In [ ]:
sb.barplot(data = data_new, x = month_year_value_counts.index , y = month_year_value_counts.values)
plt.xlabel('Year and Month Observation')
plt.ylabel('Counts of Customers')
plt.title('Year-Month vs Customer Counts')
plt.show()

In [ ]:
data_new[data_new['fecha_alta'].isna()].head(20)

We could have replaced the fecha_alta missing values using antiguedad(seniority), last purchase date(ult_fec_cli_1t), ind_nuevo(New customer Index). So we can now drop all those values as there doesn't seem a way to replace them justly.

In [ ]:
data_new['fecha_alta'].isnull().sum()

In [ ]:
data_new.dropna(subset=['fecha_alta'], inplace=True)

In [ ]:
data_new.isnull().sum()

In [ ]:
#data_NoNan.loc[data_NoNan['renta'].isnull(), 'renta'] = data_NoNan['renta'].mean()

'indrel_1mes', 'tiprel_1mes' almost denote same things. We can derive 'ult_fec_cli_1t' using the mentioned ones. If they are null, we can't generate 'ult_fec_cli_1t'. Hence, it would be wise to delete the null values of 'indrel_1mes' as there would be no way to decide their values.




In [ ]:
data_new[data_new['indrel_1mes'].isna()].head(20)

In [ ]:
data_new[data_new['indrel_1mes'].notnull()].head(20)


In [ ]:
del x

In [ ]:
data_new.dropna(subset=['indrel_1mes'], inplace=True)


In [ ]:
data_new.isnull().sum()

In [ ]:
#data_new.loc[data_new['ncodpers'] == 1535206]


codprov and nomprov contain same information. One out of them can be dropped. 


In [ ]:
data_new.drop(columns = ['cod_prov'], inplace = True)

In [ ]:
groupbyNom = data_new.groupby('nomprov')


In [ ]:
data_new.groupby(['nomprov']).size().plot(kind = "bar") 


It can be seen that most of the people belong to "Madrid, ES". So it'll be fair to replace missing ones with this as there is massive difference between the number of residents of Madrid and other places. 

In [ ]:
data_new.loc[data_new["nomprov"].isnull(), "nomprov"] = "MADRID"


In [ ]:
data_new.isnull().sum()

In [ ]:
data_new['canal_entrada'].value_counts().plot(kind = 'pie')

KHE, KFC, KAT are the channels through which most of the customers have joined the bank.

In [ ]:
lists = ['KHE', 'KFC', 'KAT']

In [ ]:
pd.crosstab(lists,data_new['segmento']).plot.bar()

In [ ]:
#grpObj = data_new.groupby(['canal_entrada', 'segmento'])

In [ ]:
#grpObj.groups.keys()

In [ ]:
Y = data_new[['segmento',  'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']].loc[data_new['segmento']=='01 - TOP']

In [ ]:
Y.drop(columns='segmento',inplace=True)


In [ ]:
Y.sum()

In [ ]:
del Y

In [ ]:
Z=data_new[['segmento',  'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',\
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',\
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',\
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',\
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',\
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',\
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',\
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']].loc[data_new['segmento']=='02 - PARTICULARES']

In [ ]:
Z.drop(columns='segmento',inplace=True)
Z.sum()

In [ ]:
del Z

In [ ]:
W=data_new[['segmento',  'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',\
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',\
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',\
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',\
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',\
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',\
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',\
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']].loc[data_new['segmento']=='03 - UNIVERSITARIO']
W.drop(columns='segmento',inplace=True)
W.sum()

In [ ]:
del W

#ind_cco_fin_ult1 , ind_recibo_ult1 , ind_cno_fin_ult1 
#out of all different categories of customers, majority of customers in each category have a current account
#VIPs having long term deposits, e-accounts, direct debit, Payroll account
#Individuals have particular account, e-account,payroll account, taxes
#University grads having payroll account, direct debit

In [ ]:
data_new.isnull().sum()

In [ ]:
data_new['nomprov'].value_counts()

In [ ]:
renta_vs_nom = data_new[['age', 'renta', 'nomprov', 'segmento']][data_new['renta'].notnull()]
renta_vs_nom

In [ ]:
sb.barplot(x = 'nomprov', y = renta_vs_nom['renta'].max(), data = renta_vs_nom)

In [ ]:
# let's groupby using segmento and nomprov
replacement = renta_vs_nom.groupby(by=['segmento', 'nomprov'],  sort=True, as_index = False, dropna = False).mean()
replacement

In [ ]:
new_renta_vs_nom = data_new[['age', 'renta', 'nomprov', 'segmento']][data_new['renta'].isna()]
new_renta_vs_nom.sort_values(by=['nomprov'], ascending=True).groupby(by=['nomprov', 'segmento'], as_index=False).mean()

In [ ]:
new_renta_vs_nom['age'][(new_renta_vs_nom['nomprov'] == 'ALAVA') & (new_renta_vs_nom['segmento'] == '01 - TOP')].mean()

In [ ]:
replacement = pd.DataFrame(replacement, columns = ['segmento', 'nomprov', 'age', 'renta'])
replacement

In [ ]:
def getRenta(age, segmento, nomprov) :
    return replacement[(replacement['nomprov'] == nomprov) & (replacement['segmento'] == segmento) & (replacement['age'] >= age)].renta

In [ ]:
print(getRenta(40, '01 - TOP', 'ALAVA'))